In [1]:
import json
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np

In [2]:
data = [json.loads(line) for line in open("concatenated_abridged.jsonl", 'r', encoding='utf-8')]
df = pd.DataFrame(data)

In [3]:
user = df.user.apply(pd.Series)
user

,id,id_str,name,screen_name,location,description,url,entities,protected,followers_count,...,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type,withheld_in_countries
0,117904481,117904481,SharonS,sfshores53,America,Conservative/#MAGA/ #WWG1WGA /MbrNRA/#Trump202...,None,{'description': {'urls': []}},False,4065,...,3C3940,True,False,False,False,False,False,False,none,NaN
1,30140760,30140760,🌊🇨🇦Lizbeth 🇨🇦🌊,ProudCdn100,Canada,Canadian 🇨🇦 Windmill Cancer Survivor. I follow...,None,{'description': {'urls': []}},False,31031,...,3E4415,True,True,False,False,False,False,False,none,NaN
2,281610530,281610530,Varney & Co.,Varneyco,New York,Airing Weekdays at 9AM ET on the Fox Business ...,https://t.co/UT25ycG2zy,{'url': {'urls': [{'url': 'https://t.co/UT25yc...,False,663539,...,333333,True,False,False,False,False,False,False,none,NaN
3,88223320,88223320,Grace Panetta,grace_panetta,"New York, NY",Politics reporter covering elections & voting ...,https://t.co/N9qln5XbI6,{'url': {'urls': [{'url': 'https://t.co/N9qln5...,False,21737,...,000000,False,True,False,False,False,False,False,none,NaN
4,44295160,44295160,SuSu Clarkson,susuclarkson,Kentucky/Florida,"wife, mother, grandmother, southerner, conserv...",None,{'description': {'urls': []}},False,893,...,362720,True,True,False,False,False,False,False,none,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140332,27780458,27780458,Average Joe Patriot,matt_aho,"Washington, USA",I’m just your average fed up & fired up Americ...,None,{'description': {'urls': []}},False,413,...,000000,False,False,False,False,False,False,False,none,NaN
140333,1081132982449119232,1081132982449119232,Tidoodoo,tidoodoo,decentralized,Connecting with you\nThank you ^-^\n\n🌎🇫🇷🐸1️⃣7️⃣,None,{'description': {'urls': []}},False,382,...,333333,True,False,True,False,False,False,False,none,NaN
140334,2290692901,2290692901,Scorpion Killer,hatenscorpions,,Making moves for change,None,{'description': {'urls': []}},False,119,...,333333,True,False,True,False,False,False,False,none,NaN
140335,1931084401,1931084401,RuthieRedSox,RuthieRedSox,New Hampshire,Retired Navy Pilot & proud Naval Academy gradu...,None,{'description': {'urls': []}},False,67360,...,333333,True,True,True,False,False,False,False,none,NaN


In [4]:
user = user[['id', 
             'name', 
             'screen_name', 
             'location', 
             'description', 
             'followers_count', 
             'friends_count', 
             'listed_count',
             'created_at',
             'favourites_count',
             'verified',
             'statuses_count',
             'lang'
            ]]

In [9]:
df = df[['created_at',
         'id',
         'full_text',
         'entities',
         'retweeted_status',
         'retweet_count',
         'favorite_count',
         'lang']]

In [10]:
my_data = pd.concat([df, user], axis=1)

# extracting original tweets

In [11]:
original_tweets = my_data[my_data['retweeted_status'].isnull()]

In [12]:
# 1. find tweets that mention Trump / Biden
# 2. get influential score
# 3. train a model for influential score prediction - user id, time of day, mention
# 4. features ingeneering for influential score prediction

In [13]:
from sklearn.preprocessing import Normalizer

In [14]:
original_tweets['retweets_norm'] = original_tweets.retweet_count.apply(lambda x: 
                                                                       (x - original_tweets.retweet_count.min())
                                                                        /(original_tweets.retweet_count.max() 
                                                                          - original_tweets.retweet_count.min()))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
original_tweets['favorite_norm'] = original_tweets.favorite_count.apply(lambda x: 
                                                                       (x - original_tweets.favorite_count.min())
                                                                        /(original_tweets.favorite_count.max() 
                                                                          - original_tweets.favorite_count.min()))

In [ ]:
original_tweets.T

# building a model to predict tweet influence

In [ ]:
X = original_tweets[['followers_count', 
                     'friends_count',
                     'listed_count',
                     'favourites_count',
                     'statuses_count',
                     ]]

In [ ]:
y = original_tweets.retweets_norm + original_tweets.favorite_norm
#y = original_tweets.retweet_count

In [ ]:
y

In [ ]:
X

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
model = RandomForestRegressor()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model_tree = DecisionTreeRegressor()

In [ ]:
model_tree.fit(X_train, y_train)

In [ ]:
y_pred_tree = model_tree.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_pred_tree, y_pred))

In [ ]:
model_tree.score(X_test, y_test)

# checking our classifier

In [ ]:
#y = original_tweets.retweets_norm + original_tweets.favorite_norm
y = original_tweets.retweets_norm

In [ ]:
y.hist()

In [ ]:
y = y.apply(lambda a: 1 if a >0.0001 else 0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
y_train

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model_clas = DecisionTreeClassifier()

In [ ]:
model_clas.fit(X_train, y_train)

In [ ]:
y_pred_clas = model_clas.predict(X_test)

In [ ]:
model_clas.score(X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, y_pred_clas)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_clas).ravel()

In [ ]:
tn, fp, fn, tp

In [ ]:
probs = model_clas.predict_proba(X_test)[:, 1]
plt.hist(probs)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve

In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
y_pred_rf = model_clas.predict_proba(X_test)[:, 1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_rf)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_rf, tpr_rf, label='RF')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()